&nbsp;
# So we wanted to play with some Public Data!
&nbsp;
## First, let's pick something to study.
&nbsp;

&nbsp;
## Let me ask you a question guys:
&nbsp;
### How many of you ever needed to find a Bench during your socially-isolated hikes?

&nbsp;
# **Tumbleweed rolls slowly**
&nbsp;

&nbsp;
## Ok, what about Toilets?
&nbsp;
## Or why not both! 
&nbsp;

&nbsp;
# Its settled, let's get some data!
&nbsp;
## **Toilet & Bench Coverage***
###### *inside an area

&nbsp;
# Imports and credentials
For this recipe :) we will need:

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
#eggs and flour
import pandas as pd, numpy as np

#butter and milk (geo libraries)
import geopandas as gpd
from shapely.geometry import Point, Polygon
#optional speed-ups
from shapely import speedups 
speedups.enable() 

#granny's kitchen-aid from the 60's 
import h3
import overpass

#whipped cream
import pydeck as pdk

#a mapbox api key for tiles (get your own at mapbox.com)
with open('./.to_ignore/keys/v1_mapbox.key', 'r') as f:
    mapbox_key= f.read()[:-1]

&nbsp;
## OpenStreetmap's Data + Overpass API
&nbsp;

#### Below is a simple method that builds a geoDataFrame from:
    * Querying OSM for all nodes/ways
    * Tagged as "amenity"with "bench" or "toilets" as a value

&nbsp;

In [ ]:
def fetch_amenity_gdf(amenity_types, area_name, adm_lvl, timeout_sec=90):
    #exports a shapefile with OSM's response to a request of data for
    #amenities of amenity_type in area_name / adm_lvl.
    #examples of adm_lvl areas: https://www.openstreetmap.org/relation/52822
    query=f'( area[admin_level={adm_lvl}]["name"="{area_name}"]; ) ->.a; \n('
    node_req='\nnode["amenity"="{0}"](area.a);\nway["amenity"="{0}"](area.a);'
    for x in amenity_types:
        query=query+node_req.format(x)
    query=query+'\n);'
    print(query)
    api = overpass.API(debug=False, timeout=timeout_sec)
    try:
        result = api.Get(query, responseformat="geojson")
        result = gpd.GeoDataFrame.from_features(result['features'])
        result.to_file('./.to_ignore/out/'+f'gdf_amenities_{area_name}_{adm_lvl}') 
    except Exception as HELL:
        print(HELL)   

&nbsp;

# Fetch & clean geoDataFrame from OSM

&nbsp;

#### With features on toilets and benches

&nbsp;

Avoid querying if file already exists

In [ ]:
area='Göteborgs Stad'
lvl='7'
try:
    gdf_amenity=gpd.read_file('./.to_ignore/out/'+f'gdf_amenities_{area}_{lvl}')
except:
    print('file_not_found, fetching amenities from OSM')
    fetch_amenity_gdf(["toilets","bench"], area,lvl)#,'Sverige','2')
    gdf_amenity=gpd.read_file('./.to_ignore/out/'+f'gdf_amenities_{area}_{lvl}')
gdf_amenity.dropna(subset=['geometry'], inplace=True) #remove nans on geometry
gdf_amenity = gdf_amenity.set_crs(epsg=4326) # this is the CRS in OSM

&nbsp;

## What features do we have here?

In [ ]:
print('Features: '+','.join(gdf_amenity.keys()))

### Woa. Cambrian explosion of Features.

&nbsp;

#### Let's remove some that we will not use today :). 

In [ ]:
    gdf_amenity=gdf_amenity[['geometry', 'amenity', 'wheelchair', 'backrest', 'fee', 'name', 'wheelcha_1', 'note']]
    gdf_amenity[:2]

&nbsp;
## Ok, lets have a look at this features in a map!

&nbsp;

#### We will use Deck's Jupyter plugin PyDeck

&nbsp;

**To enable pydeck for Jupyter, run this in command before starting the notebook:**

jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck

jupyter nbextension enable --sys-prefix --py pydeck

&nbsp;

##### And for more info on how to use deck's the jupyter integration, [check here](https://github.com/visgl/deck.gl/blob/master/bindings/pydeck/examples/geopandas_integration.py)


In [ ]:
#create separate lng and lat facilitates plotting different things
gdf_amenity['lng']= list(map(lambda p: p.x, gdf_amenity.geometry ))
gdf_amenity['lat']= list(map(lambda p: p.y, gdf_amenity.geometry ))

## First, let's generate a viewport

&nbsp;

### To make sure that basic things such as coordinates are visible

In [ ]:
# Use pydeck's data_utils module to fit a viewport to the central 70% of the data
viewport = pdk.data_utils.compute_view(points=gdf_amenity[['lng','lat']],
                                       view_proportion=0.7)

## Now, let's prepare the layers

&nbsp;

To start, I would like to just look at the geometries (points) in different colours (see docs [here](https://deck.gl/docs/api-reference/layers/scatterplot-layer))

In [ ]:
opacity=0.35
radius=60
#making toilets a bit more notable ;)
toilet_point_ly = pdk.Layer("ScatterplotLayer", getFillColor=[237,102,99],#ed6663
            data=gdf_amenity.loc[gdf_amenity.amenity=='toilets'],
            pickable=True,getRadius=radius*1.5, 
            get_position=['lng', 'lat'], opacity=opacity) 
      bench_point_ly = pdk.Layer("ScatterplotLayer", getFillColor=[78,137,174],#4e89ae
           data=gdf_amenity.loc[gdf_amenity.amenity=='bench'],
           pickable=True, getRadius=radius,
           get_position=['lng', 'lat'], opacity=opacity)

ze_map=pdk.Deck([toilet_point_ly, bench_point_ly], 
                initial_view_state=viewport, mapbox_key=mapbox_key)

In [ ]:
ze_map.show()

&nbsp;

##### If you are curious about _what else_ is well documented in OpenStreetMap, check out [overpass-turbo](http://overpass-turbo.eu/) and the [overpass python wrapper](https://github.com/mvexel/overpass-api-python-wrapper/tree/master/examples).

&nbsp;